In [1]:
bucket_name = "banpu-interns-data-exchange"
file_path = "h5/test_8.tar.gz"

In [2]:
import boto3

# Use AWS CLI named profile
session = boto3.Session(profile_name="bivlab")

In [3]:
# Upload model file to S3
s3 = session.client("s3")
s3.upload_file(file_path, bucket_name, file_path)

In [4]:
import json

# Create SageMaker execution role
iam = session.client("iam")
role_name = "SageMakerRole"

assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "sagemaker.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

try:
    create_role_response = iam.create_role(
        RoleName=role_name,
        Description="Allows SageMaker to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_document)
    )

    role_arn = create_role_response["Role"]["Arn"]
except iam.exceptions.EntityAlreadyExistsException:
    role_arn = iam.get_role(RoleName=role_name)["Role"]["Arn"]

# Attach AmazonSageMakerFullAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/AmazonSageMakerFullAccess"

try:
    iam.attach_role_policy(
        RoleName=role_name,
        PolicyArn=policy_arn
    )
except iam.exceptions.PolicyAlreadyExistsException:
    pass

# Attach AmazonS3FullAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/AmazonS3FullAccess"

try:
    iam.attach_role_policy(RoleName=role_name, PolicyArn=policy_arn)
except iam.exceptions.PolicyAlreadyExistsException:
    pass

In [5]:
import sagemaker
from sagemaker.session import Session

sagemaker_session = Session(boto_session=session)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ailama/.config/sagemaker/config.yaml


In [6]:
from sagemaker.tensorflow.serving import TensorFlowModel

model_data = "s3://" + bucket_name + "/" + file_path

model = TensorFlowModel(
    role=role_arn,
    model_data=model_data,
    framework_version="2.14",
    sagemaker_session=sagemaker_session,
)

serverless_config = sagemaker.serverless.ServerlessInferenceConfig(
    memory_size_in_mb=2048,  # Adjust based on model's needs
    max_concurrency=10,
)

predictor = model.deploy(serverless_inference_config=serverless_config)

--------------*

UnexpectedStatusException: Error hosting endpoint tensorflow-inference-2024-07-07-21-16-57-580: Failed. Reason: Received server error (0) from model with message "An error occurred while handling request as the model process exited.". See https://ap-southeast-1.console.aws.amazon.com/cloudwatch/home?region=ap-southeast-1#logEventViewer:group=/aws/sagemaker/Endpoints/tensorflow-inference-2024-07-07-21-16-57-580 in account 625531495787 for more information.. Try changing the instance type or reference the troubleshooting page https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference-troubleshooting.html